# Example for lfie cycle assessment module of AeroMAPS


## Load and process

First, the user has to load the framework and generate a process.

In [1]:
%matplotlib widget
import pandas as pd
from aeromaps.core.process import create_process
from aeromaps.core.models import (
    models_traffic,
    models_efficiency_top_down,
    models_energy_without_fuel_effect,
    models_offset,
    models_climate_simple_gwpstar,
    models_sustainability,
)
from aeromaps.models.impacts.life_cycle_assessment.life_cycle_assessment import LifeCycleAssessment

In [2]:
extended_models = {
    "models_traffic": models_traffic,
    "models_efficiency_top_down": models_efficiency_top_down,
    "models_energy_without_fuel_effect": models_energy_without_fuel_effect,
    "models_offset": models_offset,
    "models_climate_simple_gwpstar": models_climate_simple_gwpstar,
    "models_sustainability": models_sustainability,
    "life_cycle_assessment": LifeCycleAssessment(
        "life_cycle_assessment"
    ),
}

[WARNING] Db Foreground DB was here. Reseting it


Initial setup already done, skipping


[WARNING] [ParamRegistry] Param share_efuel was already defined in 'Foreground DB' : overriding.
[WARNING] Activity with name 'production' defined multiple times. Adding suffix increments to labels.


In [3]:
process = create_process(configuration_file="data_files/config.json", models=extended_models)

In [4]:
#process.list_float_inputs()

## Compute

Once all the parameters have been set up, the user can compute.

In [5]:
process.compute()

[INFO] Db changed recently, clearing cache expr
[INFO] Db changed recently, clearing cache lcia
<lambdifygenerated-1>:2: RuntimeWarning: invalid value encountered in divide
  return 1.0*total_energy*(share_efuel*(1.0*amount_elec_h2*(0.000535521699731219*electricity_switch_param_pv + 9.4484276647724e-5*electricity_switch_param_wind) + 0.00805363424130727*amount_h2/energy_density_h2) + 0.0154851332096284*(1 - share_efuel)/energy_density_kero) + 86398290.5804605*n_airports/airport_lifetime + 50494.2679161095*n_aircraft/aircraft_lifetime
<lambdifygenerated-2>:2: RuntimeWarning: invalid value encountered in divide
  return 1.0*total_energy*(share_efuel*(1.0*amount_elec_h2*(0.0827856547912274*electricity_switch_param_pv + 0.0151430617684252*electricity_switch_param_wind) + 11.7630463051155*amount_h2/energy_density_h2) + 4.04603693052418*(1 - share_efuel)/energy_density_kero) + 17708161505.3826*n_airports/airport_lifetime + 7873123.20721096*n_aircraft/aircraft_lifetime
<lambdifygenerated-2>:2

## Results

The user can then display the results. The user has access to float outputs but also to annual data outputs, with the possibility of choosing the output.

In [6]:
process.data['vector_outputs']

,acidification - accumulated exceedance (AE)[mol H+-Eq],annual_growth_rate_freight,annual_growth_rate_passenger,annual_growth_rate_passenger_long_range,annual_growth_rate_passenger_medium_range,annual_growth_rate_passenger_short_range,ask,ask_dropin_fuel,ask_electric,ask_hydrogen,ask_long_range,ask_long_range_dropin_fuel,ask_long_range_dropin_fuel_share,ask_long_range_electric,ask_long_range_electric_share,ask_long_range_hydrogen,ask_long_range_hydrogen_share,ask_medium_range,ask_medium_range_dropin_fuel,ask_medium_range_dropin_fuel_share,ask_medium_range_electric,ask_medium_range_electric_share,ask_medium_range_hydrogen,ask_medium_range_hydrogen_share,ask_short_range,ask_short_range_dropin_fuel,ask_short_range_dropin_fuel_share,ask_short_range_electric,ask_short_range_electric_share,ask_short_range_hydrogen,ask_short_range_hydrogen_share,biofuel_atj_efficiency,biofuel_atj_emission_factor,biofuel_atj_share,biofuel_ft_efficiency,biofuel_ft_msw_emission_factor,biofuel_ft_msw_share,biofuel_ft_others_emission_factor,biofuel_ft_others_share,biofuel_hefa_fog_emission_factor,biofuel_hefa_fog_share,biofuel_hefa_fuel_efficiency,biofuel_hefa_oil_efficiency,biofuel_hefa_others_emission_factor,biofuel_hefa_others_share,biofuel_mean_emission_factor,biofuel_share,biomass_atj_consumption,biomass_consumption,biomass_ft_consumption,biomass_hefa_fog_consumption,biomass_hefa_others_consumption,carbon_offset,carbon_offset_baseline_level_vs_2019,climate change - global warming potential (GWP100)[kg CO2-Eq],co2_emissions_2019technology,co2_emissions_2019technology_baseline3,co2_emissions_freight,co2_emissions_including_aircraft_efficiency,co2_emissions_including_energy,co2_emissions_including_load_factor,co2_emissions_including_operations,co2_emissions_long_range,co2_emissions_medium_range,co2_emissions_passenger,co2_emissions_per_rpk,co2_emissions_per_rtk,co2_emissions_short_range,co2_per_energy_mean,cumulative_carbon_offset,cumulative_co2_emissions,cumulative_co2_emissions_2019technology,cumulative_co2_emissions_2019technology_baseline3,cumulative_co2_emissions_including_energy,cumulative_co2_emissions_including_load_factor,...,energy_per_ask_long_range_dropin_fuel,energy_per_ask_long_range_dropin_fuel_gain,energy_per_ask_long_range_electric,energy_per_ask_long_range_hydrogen,energy_per_ask_mean,energy_per_ask_mean_without_operations,energy_per_ask_medium_range_dropin_fuel,energy_per_ask_medium_range_dropin_fuel_gain,energy_per_ask_medium_range_electric,energy_per_ask_medium_range_hydrogen,energy_per_ask_short_range_dropin_fuel,energy_per_ask_short_range_dropin_fuel_gain,energy_per_ask_short_range_electric,energy_per_ask_short_range_hydrogen,energy_per_ask_without_operations_long_range_dropin_fuel,energy_per_ask_without_operations_long_range_electric,energy_per_ask_without_operations_long_range_hydrogen,energy_per_ask_without_operations_medium_range_dropin_fuel,energy_per_ask_without_operations_medium_range_electric,energy_per_ask_without_operations_medium_range_hydrogen,energy_per_ask_without_operations_short_range_dropin_fuel,energy_per_ask_without_operations_short_range_electric,energy_per_ask_without_operations_short_range_hydrogen,energy_per_rtk_freight_dropin_fuel,energy_per_rtk_freight_electric,energy_per_rtk_freight_hydrogen,energy_per_rtk_mean,energy_per_rtk_mean_without_operations,energy_per_rtk_without_operations_freight_dropin_fuel,energy_per_rtk_without_operations_freight_electric,energy_per_rtk_without_operations_freight_hydrogen,fuel_effect_correction_contrails,hydrogen_coal_ccs_share,hydrogen_coal_share,hydrogen_electrolysis_emission_factor,hydrogen_electrolysis_share,hydrogen_gas_ccs_share,hydrogen_gas_share,hydrogen_mean_emission_factor,kerosene_emission_factor,kerosene_share,land use - soil quality index[dimensionless],level_carbon_offset,liquefaction_efficiency,load_factor,operations_contrails_gain,operations_contrails_overconsumption,operations_gain,reference_annual_growth_rate_aviation,relative_energy_per_ask_electric_wrt_dr

# Draft

In [23]:
from functools import wraps

def kwargs_decorator():
    def wrapper(f):
        @wraps(f)
        def inner_wrapper(*args, **kwargs):
            dict_kwargs = args[0].lca_params
            new_kwargs = {**dict_kwargs, **kwargs}
            return f(*args, **new_kwargs)
        return inner_wrapper
    return wrapper

class Test:
    def __init__(self):
        self.lca_params = {'a': 1.0}
        
    @kwargs_decorator()
    def foo(self, **kwargs):
        print(kwargs)

In [24]:
test = Test()
test.foo()

{'a': 1.0}


In [54]:
from functools import wraps
import lca_algebraic as agb

def kwargs_decorator(dict_kwargs):
    def wrapper(f):
        @wraps(f)
        def inner_wrapper(*args, **kwargs):
            # dict_kwargs = args[0].my_dict  # args[0] refer to 'self'
            new_kwargs = {**dict_kwargs, **kwargs}
            return f(*args, **new_kwargs)
        return inner_wrapper
    return wrapper


def args_decorator(list_args):
    def wrapper(f):
        @wraps(f)
        def inner_wrapper(*args, **kwargs):
            # dict_kwargs = args[0].my_dict  # args[0] refer to 'self'
            new_args = {*args, *list_args}
            return f(*new_args, **kwargs)
        return inner_wrapper
    return wrapper


class Test:
        
    @args_decorator(agb.all_params().keys())
    def foo(self, *args, **kwargs):
        return args
    
    
test = Test()
test.__init__()
res = test.foo()
res

('energy_density_kero',
 'electricity_switch_param',
 'amount_h2',
 'total_energy',
 'n_airports',
 'share_efuel',
 'energy_density_h2',
 'amount_elec_h2',
 'n_aircraft',
 'airport_lifetime')

In [56]:
from inspect import getfullargspec

getfullargspec(test.foo)

FullArgSpec(args=[], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [60]:
list(agb.all_params().keys())

['aircraft_lifetime',
 'n_aircraft',
 'airport_lifetime',
 'n_airports',
 'total_energy',
 'share_efuel',
 'energy_density_kero',
 'amount_elec_h2',
 'electricity_switch_param',
 'energy_density_h2',
 'amount_h2']

In [ ]:
class Test:
    def init(self):
        self.my_list = ['a', 'b']
    
    def foo(self, *args, **kwargs):
        
        
        def foo2(
        
        return args
    

In [9]:
test = ['q', 1.0]

any(isinstance(elem, str) for elem in test)

False